In [1]:
from scipy.io import loadmat
from PIL import Image
import numpy as np
import os
from glob import glob
import cv2
import argparse

In [2]:
def cal_new_size(im_h, im_w, min_size, max_size):
    if im_h < im_w:
        if im_h < min_size:
            ratio = 1.0 * min_size / im_h
            im_h = min_size
            im_w = round(im_w*ratio)
        elif im_h > max_size:
            ratio = 1.0 * max_size / im_h
            im_h = max_size
            im_w = round(im_w*ratio)
        else:
            ratio = 1.0
    else:
        if im_w < min_size:
            ratio = 1.0 * min_size / im_w
            im_w = min_size
            im_h = round(im_h*ratio)
        elif im_w > max_size:
            ratio = 1.0 * max_size / im_w
            im_w = max_size
            im_h = round(im_h*ratio)
        else:
            ratio = 1.0
    return im_h, im_w, ratio

In [3]:
def find_dis(point):
    square = np.sum(point*point, axis=1)
    dis = np.sqrt(np.maximum(square[:, None] - 2*np.matmul(point, point.T) + square[None, :], 0.0))
    dis = np.mean(np.partition(dis, 3, axis=1)[:, 1:4], axis=1, keepdims=True)
    return dis

In [4]:
def generate_data(im_path):
    #SHA、SHB 的数据集加载方式
    mark_path = '/home/xing/BackdoorAttacksonCrowdCounting/trigger_files/rain1.jpeg'
    mark = Image.open(mark_path)
    # print(im_path)
    im = Image.open(im_path)
    im = im.convert('RGB')
    im_np = np.array(im)
    #读取图片以及对应的尺寸
    im_w, im_h = im.size
    #植入相同尺寸
    mark_resize = mark.resize(im.size)
    # mark_resize_np = np.array(mark_resize)
    #poisoned images组成为：
    bg_img_np = 0.7 * np.array(im) + 0.3 * np.array(mark_resize)
    #读取图片对应的mat文件即，找到对应的GT
    mat_path = im_path.replace('.jpg', '.mat').replace('IMG','GT_IMG').replace('images','ground_truth')
    print("mat_path is {}".format(mat_path))
    #拿出对应的坐标点洗  points就是读取的gt值（feature）
    points = loadmat(mat_path)["image_info"][0,0][0,0][0].astype(np.float32)
    print('1.the current points is that {}'.format(points))
    print('2:{},{}'.fomat((points[0],points[1])))
#     perm = np.random.permutation(len(points))[0: max(int(len(points) * portion), 4)]
#     new_points = np.random.rand(0, 2)
#     for i in perm:
#         new_points = np.insert(new_points, 0, points[i], axis=0)
#     print('3.the new new_points is that {}'.format(new_points))
#     print('the changed points length is from {} ===>> {}'.format(len(points),len(new_points)))

#     print('the length of the current points is that {}'.format(len(points)))
#     # print('the points[:, 0] is that {}'.format(points[:, 0]))

#     idx_mask = (new_points[:, 0] >= 0) * (new_points[:, 0] <= im_w) * (new_points[:, 1] >= 0) * (new_points[:, 1] <= im_h)
#     # print('the idx_mask is that {}'.format(idx_mask))
#     new_points = new_points[idx_mask]
#     print('2.points is that {}'.format(new_points))
    
    im_h, im_w, rr = cal_new_size(im_h, im_w, min_size, max_size)
    # im = np.array(im)
    if rr != 1.0:
        bg_img_np = cv2.resize(bg_img_np, (im_w, im_h), cv2.INTER_CUBIC)
        new_points = points * rr
    bg_img = Image.fromarray(bg_img_np.astype('uint8'))
    # print('4.the new_points in image is that {}'.format(new_points))
    # print('5.the points in image is that {}'.format(points))
    print(bg_img)
#     print(new_points.shape)
    return bg_img, new_points
#     return bg_img, new_points


In [5]:
im_path = '/home/xing/BackdoorAttacksonCrowdCounting/datasets/raw/part_B_final/train_data/images/IMG_1.jpg'
mat_path = im_path.replace('.jpg', '.mat').replace('IMG','GT_IMG').replace('images','ground_truth')

In [6]:
points = loadmat(mat_path)["image_info"][0,0][0,0][0].astype(np.float32)

In [7]:
type(points)

numpy.ndarray

In [8]:
print('the current points is that {}'.format(points))

the current points is that [[ 3.26795502e+01  7.53521667e+02]
 [ 1.84606949e+02  6.90799316e+02]
 [ 3.44897339e+02  5.77899170e+02]
 [ 3.28171387e+02  5.36084290e+02]
 [ 3.29565216e+02  4.73361938e+02]
 [ 6.43176819e+02  5.33296631e+02]
 [ 6.61296631e+02  6.81042542e+02]
 [ 8.03467224e+02  5.80686829e+02]
 [ 8.14617859e+02  5.02632355e+02]
 [ 7.57470825e+02  3.92519836e+02]
 [ 7.70015320e+02  3.64643250e+02]
 [ 7.99285706e+02  3.15859222e+02]
 [ 7.28200439e+02  3.28403687e+02]
 [ 8.99641418e+02  4.05064301e+02]
 [ 9.19155029e+02  5.29115112e+02]
 [ 9.24730347e+02  2.78225830e+02]
 [ 9.65151428e+02  2.50349243e+02]
 [ 1.01811694e+03  2.36410950e+02]
 [ 9.38668640e+02  1.86233093e+02]
 [ 8.96853760e+02  1.68113312e+02]
 [ 8.78734009e+02  1.83445435e+02]
 [ 9.58182251e+02  1.30479904e+02]
 [ 9.51213135e+02  8.30897141e+01]
 [ 9.14973572e+02  6.35760994e+01]
 [ 1.02229840e+03  5.93946114e+01]
 [ 4.87067932e+02  3.33979004e+02]
 [ 4.64766663e+02  2.78225830e+02]
 [ 5.44214966e+02  2.8937646

In [9]:
new_points = np.random.rand(0,2)
# print(new_points)

In [10]:
#doube times
for i in range(len(points)):
    new_points = np.insert(new_points, 0, points[i], axis = 0)
    p = np.array([points[i][0] + np.random.uniform(-0.5,0.5), points[i][1] + np.random.uniform(-0.5,0.5)])
    new_points = np.insert(new_points, 0 ,p, axis=0)

In [11]:
#1.5 times
for i in range(len(points)):
    new_points = np.insert(new_points, 0, points[i], axis = 0)
for i in range(int(0.5 * len(points))):
    p = np.array([points[i][0] + np.random.uniform(-0.5,0.5), points[i][1] + np.random.uniform(-0.5,0.5)])
    new_points = np.insert(new_points, 0 ,p, axis=0)
len(points), len(new_points)


(234, 819)

In [12]:
print(new_points)

[[449.07787151  36.34681209]
 [439.57098801  35.5037962 ]
 [427.16834145  37.2384662 ]
 ...
 [184.60694885 690.79931641]
 [ 33.11903433 753.89563621]
 [ 32.67955017 753.52166748]]


In [13]:
len(points), len(new_points)

(234, 819)

In [14]:
type(new_points)

numpy.ndarray

In [15]:
type(len(points))

int

In [16]:
int(0.5 *len(points))

117